In [413]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import pymdp
from pymdp import utils

# O, S, U

In [414]:
# Sales states (how much has been sold)
Sales_1 =  np.array([0, 1, 2, 3, 4, 5])
Sales_2 =  np.array([0, 1, 2, 3, 4, 5])

# Market states (if I have produced enaugh or not)
Market_1 = ['Produced excess', 'Produced right', 'Not produced enaugh']
Market_2 = ['Produced excess', 'Produced right', 'Not produced enaugh']

# number of states
n_states_1 = [len(Sales_1), len(Market_1)]
n_states_2 = [len(Sales_2), len(Market_2)]

# number of factors
n_factors_1 = len(n_states_1) 
n_factors_2 = len(n_states_2)

# actions
Production_1 =  np.array([0, 1, 2, 3, 4, 5])
Production_2 =  np.array([0, 1, 2, 3, 4, 5])

satisfy_market_action = ['Market survey'] # OK, si potrebbe aggiungere la scelta di non fare il market survey, magari farlo ha costo, prima però facciamo
                                          # funzionare il caso base

num_controls_1 = [len(Production_1), len(satisfy_market_action)]
num_controls_2 = [len(Production_2), len(satisfy_market_action)]

# observations of sales and market observations (satisfied costumer demand or not)
obs_sales_1 = np.array([0, 1, 2, 3, 4, 5])
obs_sales_2 = np.array([0, 1, 2, 3, 4, 5])

obs_market_1 = ['Exceded', 'Satisfied', 'Unsatisfied'] 
obs_market_2 = ['Exceded', 'Satisfied', 'Unsatisfied']

# number of observations and modalities
num_obs_1 = [len(obs_sales_1), len(obs_market_1)]
num_obs_2 = [len(obs_sales_2), len(obs_market_2)]

num_modalities_1 = len(num_obs_1) 
num_modalities_2 = len(num_obs_2)

# A

In [415]:
import numpy as np

# Definizione della dimensione della matrice
n = 5  # Puoi cambiare la dimensione a piacere

# Creazione di una matrice di zeri
matrix = np.zeros((n, n), dtype=int)

# Riempimento della prima sottodiagonale con 1
for i in range(1, n):
    matrix[i, i-1] = 1

# Sostituzione dell'ultimo elemento in basso a destra con 1
matrix[-1, -1] = 1

# Stampa della matrice
print(matrix)

[[0 0 0 0 0]
 [1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]]


In [416]:
A1 = utils.obj_array(num_modalities_1)
A2 = utils.obj_array(num_modalities_2)

## sales

In [417]:
# Sales obs x State Sales x State Market
A1_sales = np.zeros((len(obs_sales_1), len(Sales_1), len(Market_1)))
A2_sales = np.zeros((len(obs_sales_2), len(Sales_2), len(Market_2)))

# Agent 1 Likelihood
for i in range(1, len(Sales_1)):
    A1_sales[i-1, i, 0] = 1
A1_sales[0, 0, 0] = 1

A1_sales[:,:,1] = np.eye(len(Sales_1)) # match demand 

for i in range(1, len(Sales_1)):
    A1_sales[i, i-1, 2] = 1
A1_sales[-1, -1, 2] = 1

# Agent 2 Likelihood
for i in range(1, len(Sales_2)):
    A2_sales[i-1, i, 0] = 1
A2_sales[0, 0, 0] = 1

A2_sales[:,:,1] = np.eye(len(Sales_2)) # match demand 

for i in range(1, len(Sales_2)):
    A2_sales[i, i-1, 2] = 1
A2_sales[-1, -1, 2] = 1

A1[0] = A1_sales
A2[0] = A2_sales

## market

In [418]:
A1_market = np.zeros((len(obs_market_1), len(Sales_1), len(Market_1)))
A2_market = np.zeros((len(obs_market_2), len(Sales_2), len(Market_2)))

trust_market = 1.0 # 1 means perfect trust
# NB: per come sono fatte le cose al momento non ha senso !=1

# Assign perception probabilities
A1_market[0, :, 0] = trust_market           # exceded <- produced excess
A1_market[1, :, 0] = (1 - trust_market)/2   # satisfied <- produced excess
A1_market[2, :, 0] = (1 - trust_market)/2   # etc...
A1_market[0, :, 1] = (1 - trust_market)/2
A1_market[1, :, 1] = trust_market
A1_market[2, :, 1] = (1 - trust_market)/2
A1_market[0, :, 2] = (1 - trust_market)/2
A1_market[1, :, 2] = (1 - trust_market)/2
A1_market[2, :, 2] = trust_market

A2_market[0, :, 0] = trust_market
A2_market[1, :, 0] = (1 - trust_market)/2
A2_market[2, :, 0] = (1 - trust_market)/2
A2_market[0, :, 1] = (1 - trust_market)/2
A2_market[1, :, 1] = trust_market
A2_market[2, :, 1] = (1 - trust_market)/2
A2_market[0, :, 2] = (1 - trust_market)/2
A2_market[1, :, 2] = (1 - trust_market)/2
A2_market[2, :, 2] = trust_market

A1[1] = A1_market
A2[1] = A2_market

# B

In [419]:
B1 = utils.obj_array(n_factors_1)
B2 = utils.obj_array(n_factors_2)

# sales

In [420]:
B1_sales = np.zeros((len(Sales_1), len(Sales_1), len(Production_1)))
B2_sales = np.zeros((len(Sales_2), len(Sales_2), len(Production_2)))

#I can sell as much as I produce (potentially) independently from previous sales
for i in range(len(Production_1)):
    for end_state in range(len(Production_1)):
        if end_state <= i:
            B1_sales[end_state, :, i] = 1/(i+1)

#completing agent2 transition matrix
for i in range(len(Production_2)):
    for end_state in range(len(Production_2)):
        if end_state <= i:
            B2_sales[end_state, :, i] = 1/(i+1)

B1[0] = B1_sales
B2[0] = B2_sales

## market

In [421]:
B1_market = np.zeros((len(Market_1), len(Market_1), len(satisfy_market_action)))
B2_market = np.zeros((len(Market_2), len(Market_2), len(satisfy_market_action)))

B1_market[:,:,0] = np.full((len(Market_1), len(Market_1)), 1/3)
B2_market[:,:,0] = np.full((len(Market_1), len(Market_1)), 1/3)

B1[1] = B1_market
B2[1] = B2_market

# C

In [422]:
T_step = 5

C1 = np.array([np.zeros((n, T_step)) for n in num_obs_1], dtype=object)
C2 = utils.obj_array_zeros(num_obs_2)

## Profit

In [423]:
assumed_market_price_1 = 10 # earning from a single piece
production_cost_1 = 5 # price of a single piece
discount_rate_1 = np.ones(len(Production_1)) #np.array([1, 1, 0.9, 0.8, 0.7, 0.6]) # the more I produce the less I pay

assumed_market_price_2 = 10 # earning from a single piece
production_cost_2 = 5 # price of a single piece
discount_rate_2 = np.ones(len(Production_2)) # the more I produce the less I pay

In [424]:
def gain(market_price, prod_cost, n_sells, Production):
    C_gain = (market_price-prod_cost)*Production - np.maximum(0,(Production-n_sells))*prod_cost
    return C_gain

In [425]:
C_gain_1 = gain(market_price=assumed_market_price_1, prod_cost=production_cost_1, n_sells=max(Production_1), Production=Production_1)
C_gain_2 = gain(market_price=assumed_market_price_2, prod_cost=production_cost_2, n_sells=max(Production_2), Production=Production_2)

C1[0][:,0] = C_gain_1
C2[0] = C_gain_2

## market

In [426]:
C1_market = np.zeros(len(obs_market_1))
C1[1][:,0] = C1_market

C2_market = np.zeros(len(obs_market_2))
C2[1] = C2_market

# D

In [427]:
D1 = utils.obj_array(n_factors_1)
D2 = utils.obj_array(n_factors_2)

# Sales state prior
D1[0] = np.full(len(Sales_1), 1 / len(Sales_1))
D2[0] = np.full(len(Sales_2), 1 / len(Sales_2))

# Market state prior
D1[1] = np.full(len(Market_1), 1 / len(Market_1))
D2[1] = np.full(len(Market_2), 1 / len(Market_2))

# Agent

In [428]:
B1_prior = utils.dirichlet_like(B1)
B2_prior = utils.dirichlet_like(B2)

In [429]:
from pymdp.agent import Agent

my_agent1 = Agent(A = A1, B = B1, C = C1, D = D1, policy_len=3,
                    use_states_info_gain = False, #True di default
                    use_param_info_gain = False, #False di default
                    num_controls = num_controls_1, 
                    pA = None, lr_pA=1.0, modalities_to_learn="all",
                    pB = B1_prior, lr_pB=0.5, pD = None, lr_pD=1.0, factors_to_learn='all')

my_agent2 = Agent(A = A2, B = B2, C = C2, D = D2, policy_len=3,
                    use_states_info_gain = False, #True di default
                    use_param_info_gain = False, #False di default
                    num_controls = num_controls_2, 
                    pA = None, lr_pA=1.0, modalities_to_learn="all",
                    pB = B1_prior, lr_pB=0.5, pD = None, lr_pD=1.0, factors_to_learn='all')

# Environment

In [430]:
class CournotMarket(object):

    def __init__(self, reputation=0.5, total_demand=5, max_customer_price=10, sensibility=1):
        self.reputation = float(reputation)
        self.total_demand = float(total_demand)
        self.max_customer_price = float(max_customer_price)
        self.sensibility = float(sensibility)
        self.market_obs_names = ['Satisfied', 'Exceded', 'Unsatisfied']
    
    def market_status(self, actual_production, preferred_demand):
        if actual_production > preferred_demand:
            return 'Exceded'
        elif actual_production < preferred_demand:
            return 'Unsatisfied'
        else:
            return 'Satisfied'

    def step(self, action1, action2):
        total_production = action1[0] + action2[0]
        
        market_price = max(0, self.max_customer_price - self.sensibility * total_production)
        #market_price = 6
        
        preferred_demand_1 = int(self.total_demand * self.reputation)
        preferred_demand_2 = self.total_demand - preferred_demand_1
        
        actual_sales_1 = min(action1[0], preferred_demand_1)
        actual_sales_2 = min(action2[0], preferred_demand_2)
        
        remaining_demand_1 = preferred_demand_1 - actual_sales_1
        remaining_demand_2 = preferred_demand_2 - actual_sales_2
        
        if remaining_demand_1 > 0 and action2[0] > actual_sales_2:
            extra_sales_2 = min(action2[0] - actual_sales_2, remaining_demand_1)
            actual_sales_2 += extra_sales_2
            remaining_demand_1 -= extra_sales_2
        
        if remaining_demand_2 > 0 and action1[0] > actual_sales_1:
            extra_sales_1 = min(action1[0] - actual_sales_1, remaining_demand_2)
            actual_sales_1 += extra_sales_1
            remaining_demand_2 -= extra_sales_1
        
        obs_sales = [int(actual_sales_1), int(actual_sales_2)] 
        obs_market = [self.market_status(action1[0], preferred_demand_1),
                      self.market_status(action2[0], preferred_demand_2)]
        return obs_sales, obs_market, market_price

# Loop

In [431]:
def run_active_inference_loop(my_agent1, my_agent2, my_env, T = 5):
    """Initialize first observation"""
    market_label = ['Exceded' ,'Satisfied', 'Unsatisfied']
    obs1 = [0, obs_market_1.index(market_label[0])] #sales at T = -1 + market satisfaction <= come se dovessimo ancora produrre
    obs2 = [0, obs_market_2.index(market_label[0])]

    for t in range(T):
        qs_1 = my_agent1.infer_states(obs1)
        qs_2 = my_agent2.infer_states(obs2)

        q_pi_1, efe_1 = my_agent1.infer_policies()
        q_pi_2, efe_2 = my_agent2.infer_policies()

        action_1 = my_agent1.sample_action()
        action_2 = my_agent2.sample_action()

        obs = my_env.step(action_1, action_2)
        obs_sales = obs[0]
        obs_market = obs[1]
        market_price = obs[2]

        my_agent1.C[0][:,t] = gain(market_price=market_price, prod_cost=production_cost_1, n_sells=obs_sales[0], Production=Production_1)
        my_agent2.C[0] = gain(market_price=market_price, prod_cost=production_cost_2, n_sells=obs_sales[1], Production=Production_2)
        
        obs1 = [obs_sales[0], obs_market_1.index(obs_market[0])]
        obs2 = [obs_sales[1], obs_market_2.index(obs_market[1])]

        print(f"C1[{t}] = {my_agent1.C[0][:, t]} market price = {market_price}")
        print("C2 = ", my_agent2.C[0], "markt price = ", market_price)
        print(f'Production industry 1 at time {t}: {action_1[0]}')
        print(f'Production industry 2 at time {t}: {action_2[0]}') 
        print(f'Sells of industries at time {t}: {obs[0]}')
        print(f'Market at time {t}: {obs[1]}')


# Main

In [432]:
import warnings
warnings.filterwarnings('ignore')

In [433]:
reputation_env = 0.5
total_demand_env = 6
env = CournotMarket(reputation = reputation_env, total_demand = total_demand_env, max_customer_price = 16, sensibility = 1)

T = T_step

run_active_inference_loop(my_agent1, my_agent2, env, T = T)

C1[0] = [ 0.  1.  2.  3. -1. -5.] market price = 6.0
C2 =  [ 0.  1.  2.  3. -1. -5.] markt price =  6.0
Production industry 1 at time 0: 5.0
Production industry 2 at time 0: 5.0
Sells of industries at time 0: [3, 3]
Market at time 0: ['Exceded', 'Exceded']
C1[1] = [ 0.  5. 10. 15. 15. 15.] market price = 10.0
C2 =  [ 0.  5. 10. 15. 15. 15.] markt price =  10.0
Production industry 1 at time 1: 3.0
Production industry 2 at time 1: 3.0
Sells of industries at time 1: [3, 3]
Market at time 1: ['Satisfied', 'Satisfied']
C1[2] = [0. 3. 6. 9. 7. 5.] market price = 8.0
C2 =  [0. 3. 6. 9. 7. 5.] markt price =  8.0
Production industry 1 at time 2: 3.0
Production industry 2 at time 2: 5.0
Sells of industries at time 2: [3, 3]
Market at time 2: ['Satisfied', 'Exceded']
C1[3] = [0. 3. 6. 9. 7. 5.] market price = 8.0
C2 =  [0. 3. 6. 9. 7. 5.] markt price =  8.0
Production industry 1 at time 3: 3.0
Production industry 2 at time 3: 5.0
Sells of industries at time 3: [3, 3]
Market at time 3: ['Satisfied